In [0]:
 !wget http://download1499.mediafire.com/vsbtrxxkbk5g/forhwmpnp888mrl/GlassOrNot_dataset.rar
 !pip install unrar
 !unrar x '/content/GlassOrNot_dataset.rar'

In [2]:
import os

print('start____________1')

from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.vgg16 import VGG16

ir = 244 #image_resize
pre_trained_model = VGG16(input_shape = (ir,ir,3), include_top = False, weights='imagenet')


start____________1


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [3]:
last_layer = pre_trained_model.get_layer('block5_pool')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

last layer output shape: (None, 7, 7, 512)


In [4]:
for layer in pre_trained_model.layers:
    layer.trainable = False
print('done')

done


In [5]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x) #de tranh overfit
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

#configure and compile the model
model = Model(pre_trained_model.input, x)
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics= ['accuracy'])
print('done')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
done


In [0]:
model.summary()

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#Data augmentation
train_datagen = ImageDataGenerator(
      rescale = 1./255,#dua du lieu ve khoang 0->1
      shear_range = 0.2,#cat ngau nhien mot goc theo chiu kim dong ho
      zoom_range = 0.2,
      horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
    '/content/GlassOrNot/TrainingSet',
    target_size=(ir,ir),
    batch_size=2,
    class_mode='binary'#bieu dien thanh kieu binary
)
test_set = test_datagen.flow_from_directory(
    '/content/GlassOrNot/ValidationSet',
     target_size=(ir,ir),
    batch_size=2,
    class_mode='binary'
)

Found 200 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [8]:
from IPython.display import display
from PIL import Image
from tensorflow.keras.optimizers import RMSprop

# Configure and compile the model
model = Model(pre_trained_model.input, x)
model.compile(loss='binary_crossentropy',
              metrics=['acc'])

model.fit_generator(
      training_set,
      epochs=2,
      validation_data=test_set,
      verbose=1)

Epoch 1/2
100/100 [==============================] - 12s 116ms/step - loss: 4.6403 - acc: 0.5600 - val_loss: 0.9582 - val_acc: 0.7500
Epoch 2/2
100/100 [==============================] - 9s 89ms/step - loss: 1.3438 - acc: 0.8000 - val_loss: 2.2359 - val_acc: 0.7500


In [16]:
import os,random
from keras.preprocessing import image
from tensorflow.keras.models import load_model
import numpy as  np
ir=244 #image size

model = load_model('/content/drive/My Drive/Colab Notebooks/VGG_pretrained_80')

path = '../content/TestOfGlassOrNot/'

imagelist = os.listdir(path)
print(len(imagelist))
total = 0

y_true = []
y_pred = []

img_list = []

while(imagelist):
    a=random.choice(imagelist)
    print(a)
    imagelist.remove(a)
    test_image = image.load_img(path+a, target_size = (ir,ir))

    img_list.append(test_image)

    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image,axis=0)
    result = model.predict(test_image)
    
    print(result[0][0])
    if( result[0][0] >=0.5):
        prediction = 'Yes'
        y_pred.append(1)
    else:
        prediction = 'No'
        y_pred.append(0)
    print('it is_______________________________:'+prediction)
    if(a.find(prediction,0,3)!=-1):
        total= total +1
        #print('TOTAL___:'+ str(total))
    if(a.find('Yes',0,3)==-1):
        y_true.append(0)
    else:
        y_true.append(1)

print('TOTAL___:'+ str(total))
print(y_pred)
print(y_true)

20
Noo127.jpg
0.9999999
it is_______________________________:Yes
Noo130.jpg
0.0012020888
it is_______________________________:No
Yes183.jpg
1.0
it is_______________________________:Yes
Noo124.jpg
0.0
it is_______________________________:No
Yes181.jpg
1.0
it is_______________________________:Yes
Yes185.jpg
1.0
it is_______________________________:Yes
Yes182.jpg
1.0
it is_______________________________:Yes
Noo125.jpg
1.0
it is_______________________________:Yes
Noo132.jpg
0.0
it is_______________________________:No
Yes179.jpg
1.0
it is_______________________________:Yes
Noo133.jpg
0.0
it is_______________________________:No
Noo128.jpg
0.0
it is_______________________________:No
Yes188.jpg
1.0
it is_______________________________:Yes
Noo131.jpg
0.0
it is_______________________________:No
Yes184.jpg
1.0
it is_______________________________:Yes
Yes180.jpg
1.0
it is_______________________________:Yes
Noo126.jpg
0.0
it is_______________________________:No
Noo129.jpg
0.0
it is_________________

In [37]:
import matplotlib.pyplot as plt

MyDict = {
    0:'No',
    1:'Yes'
}

fig=plt.figure(figsize=(20, 90))
columns = 2
rows = 10
img_index = 0
for i in range(1, columns*rows +1):
    img = img_list[img_index]
    fig.add_subplot(rows, columns, i)
    plt.title('Prediction:  ' + MyDict[y_pred[img_index]], size=15, color = 'red')
    plt.imshow(np.real(img))
    img_index+= 1
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [38]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

print('accuracy = ',accuracy_score(y_true, y_pred))
cnf_matrix = confusion_matrix(y_true, y_pred)
print(cnf_matrix)

def cm2pr_binary(cm):
    p = cm[0,0]/np.sum(cm[:,0])
    r = cm[0,0]/np.sum(cm[0])
    return (p, r)

p,r=cm2pr_binary(cnf_matrix)
print('Precision: ' + str(p))
print('Recall: ' + str(r) )

accuracy =  0.9
[[ 8  2]
 [ 0 10]]
Precision: 1.0
Recall: 0.8
